In [1]:
from __future__ import print_function
import numpy as np
import warnings
import pandas as pd 
import matplotlib.pyplot as plt
import os
import keras

# Removed: !pip install keras_applications (won't work on Kaggle)

from keras.models import Model
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from tensorflow.keras.utils import get_source_inputs
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from keras import backend as K
from tensorflow.keras.optimizers import RMSprop

from keras.applications.imagenet_utils import decode_predictions, preprocess_input

# ✅ Removed this line — it causes the error and is not needed
# from keras_applications.imagenet_utils import _obtain_input_shape

# Load metadata
df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")

# See basic structure
print(df.shape)
df.head()


2025-04-20 10:29:41.464301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 10:29:41.464410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 10:29:41.587240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(106800, 15)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [3]:
# Load metadata
df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")

# Map class labels to numeric values
label_map = {
    'seizure': 0,
    'lpd': 1,
    'gpd': 2,
    'lrda': 3,
    'grda': 4,
    'other': 5
}

# Convert text labels to lowercase (just in case) and map to label_id
df['label'] = df['expert_consensus'].str.lower()
df['label_id'] = df['label'].map(label_map)

# Check if mapping worked
df[['expert_consensus', 'label', 'label_id']].head()


,expert_consensus,label,label_id
0,Seizure,seizure,0
1,Seizure,seizure,0
2,Seizure,seizure,0
3,Seizure,seizure,0
4,Seizure,seizure,0


In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

PATH = '/kaggle/input/hms-harmful-brain-activity-classification'
dataset_path = os.listdir(PATH)

folders = os.listdir(PATH)
print (folders)  #what kinds of files are in this dataset

print("No. of Files/Folders found: ", len(dataset_path))

['train_eegs', 'sample_submission.csv', 'train_spectrograms', 'test_eegs', 'example_figures', 'test_spectrograms', 'train.csv', 'test.csv']
No. of Files/Folders found:  8


In [5]:
df = pd.read_csv(PATH+"/train.csv")

In [6]:
# Let's check how many samples for each category are present
print("Total number of data in the dataset: ", len(df))

data_count = df['expert_consensus'].value_counts()

print("expert_consensus data in each category: ")
print(data_count)

Total number of data in the dataset:  106800
expert_consensus data in each category: 
expert_consensus
Seizure    20933
GRDA       18861
Other      18808
GPD        16702
LRDA       16640
LPD        14856
Name: count, dtype: int64


In [7]:
df = df.drop(columns=['eeg_sub_id', 'eeg_label_offset_seconds', "spectrogram_sub_id", "spectrogram_label_offset_seconds","label_id",])
df=df.drop_duplicates(subset=("spectrogram_id"))
data_count = df['expert_consensus'].value_counts()

print("expert_consensus data in each category: ")
print(data_count)

expert_consensus data in each category: 
expert_consensus
Other      4699
Seizure    2045
GRDA       1422
LPD        1284
GPD        1025
LRDA        663
Name: count, dtype: int64


In [8]:
df.replace({'GRDA':'NO'}, inplace=True)
df.replace({'Other':'NO'}, inplace=True)
df.replace({'GPD':'NO'}, inplace=True)
df.replace({'LRDA':'NO'}, inplace=True)
df.replace({'LPD':'NO'}, inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y = df['expert_consensus'].values

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

[1 0 0 ... 0 0 0]


In [10]:
# Assuming y is our target variable as a 1D array or DataFrame column
y = y.reshape(-1, 1)  # Reshape to column vector 
# Create OneHotEncoder object with specified categories
onehotencoder = OneHotEncoder(categories='auto')  # 'auto' automatically determines categories
Y = onehotencoder.fit_transform(y).toarray()
print(Y.shape)  # This should print the shape of the one-hot encoded output


(11138, 2)


In [12]:
# Load spectrogram data from Parquet file
ecg_spectrogram = pd.read_parquet(PATH+'/train_spectrograms/353733.parquet')

In [13]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications.vgg16 import preprocess_input

# Resizing and Preprocessing Spectrogram Images
def preprocess_spectrogram(spectrogram, target_size=(150, 150)):
    # Normalizing spectrogram data to [0, 255] range 
    if spectrogram.dtype == np.float64:
        spectrogram = (spectrogram - spectrogram.min()) / (spectrogram.max() - spectrogram.min()) * 255.0
        spectrogram = spectrogram.astype(np.uint8)

    # Resize spectrogram to target size using OpenCV
    resized_spectrogram = cv2.resize(spectrogram, target_size)
    # Convert to RGB image (3 channels)
    rgb_spectrogram = cv2.cvtColor(resized_spectrogram, cv2.COLOR_GRAY2RGB)
    # Convert to array and preprocess according to VGG-16 requirements
    preprocessed_spectrogram = img_to_array(rgb_spectrogram)
    preprocessed_spectrogram = preprocess_input(preprocessed_spectrogram)
    return preprocessed_spectrogram

# Preprocess first spectrogram in the DataFrame
first_spectrogram = ecg_spectrogram.iloc[0, :].values  #each row represents a spectrogram
preprocessed_spectrogram = preprocess_spectrogram(first_spectrogram)
print(preprocessed_spectrogram.shape)

(150, 150, 3)


In [14]:
import os
path='/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
images=[]
# Ignore all the warnings temporarily
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for f in os.listdir(path):
        file_path=path+f
        df = pd.read_parquet(file_path)
        first_spectrogram = df.iloc[0, :].values  # Assuming each row represents a spectrogram
        preprocessed_spectrogram = preprocess_spectrogram(first_spectrogram)
        images.append(preprocessed_spectrogram)

In [15]:
images = np.array(images)
images.shape

(11138, 150, 150, 3)

In [16]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=42)

#inspect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(8910, 150, 150, 3)
(8910, 2)
(2228, 150, 150, 3)
(2228, 2)


In [28]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/vgg/tensorflow2/default/1/model.h5')


In [29]:
from sklearn.metrics import confusion_matrix
#Predictions on the test set
pred_y = model.predict(test_x)

2025-04-20 10:48:30.950356: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.18794, expected 3.54228
2025-04-20 10:48:30.950414: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 5.14647, expected 4.50081
2025-04-20 10:48:30.950424: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 5.30074, expected 4.65509
2025-04-20 10:48:30.950432: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 4.91575, expected 4.27009
2025-04-20 10:48:30.950440: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 10: 3.57367, expected 2.92801
2025-04-20 10:48:30.950447: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 4.05899, expected 3.41334
2025-04-20 10:48:30.950455: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 5.27592, expected 4.63026
2025-04-20 10:48:30.950463: E external/local_xla/xla

 7/70 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step

I0000 00:00:1745146119.411127     132 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1745146119.426812     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


69/70 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

2025-04-20 10:48:42.051758: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22500: 4.05562, expected 3.31202
2025-04-20 10:48:42.051808: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22501: 5.54755, expected 4.80395
2025-04-20 10:48:42.051817: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22502: 5.65566, expected 4.91206
2025-04-20 10:48:42.051825: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22503: 5.19665, expected 4.45304
2025-04-20 10:48:42.051833: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22504: 5.57293, expected 4.82933
2025-04-20 10:48:42.051843: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22505: 5.45659, expected 4.71299
2025-04-20 10:48:42.051850: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 22507: 6.10612, expected 5.36251
2025-04-20 10:48:42.051858:

70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 124ms/step


W0000 00:00:1745146127.976934     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [30]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

 5/70 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8400 - loss: 0.4422

W0000 00:00:1745146216.092594     133 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.8282 - loss: 0.4595
Loss = 0.46824759244918823
Test Accuracy = 0.8222621083259583


W0000 00:00:1745146219.034384     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [31]:
from sklearn.metrics import confusion_matrix
#Predictions on the test set
pred_y = model.predict(test_x)

70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


In [33]:
new_model = tf.keras.models.load_model('/kaggle/input/resnet/tensorflow2/default/1/efficientnet_model.h5')

In [34]:
pred_y = new_model.predict(test_x)

70/70 ━━━━━━━━━━━━━━━━━━━━ 15s 113ms/step


In [35]:
preds = new_model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.7883 - loss: 1.6130
Loss = 1.7389264106750488
Test Accuracy = 0.7836624979972839


In [36]:
vgg19 = tf.keras.models.load_model('/kaggle/input/vg19/tensorflow2/default/1/vgg19_model.h5')

In [38]:
pred_y = vgg19.predict(test_x)

 5/70 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step

W0000 00:00:1745147417.231011     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step


W0000 00:00:1745147420.464057     133 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [40]:
preds = vgg19.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

 3/70 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.8438 - loss: 0.4367

W0000 00:00:1745147445.665138     133 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.8282 - loss: 0.4593
Loss = 0.4681060016155243
Test Accuracy = 0.8222621083259583


W0000 00:00:1745147449.095024     133 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [41]:
rsmodel = tf.keras.models.load_model('/kaggle/input/originalresnet/tensorflow2/default/1/resnet152_model.h5')

In [43]:
pred_y = rsmodel.predict(test_x)

70/70 ━━━━━━━━━━━━━━━━━━━━ 24s 195ms/step


In [45]:
preds = rsmodel.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

70/70 ━━━━━━━━━━━━━━━━━━━━ 18s 117ms/step - accuracy: 0.8282 - loss: 0.4584
Loss = 0.46903178095817566
Test Accuracy = 0.8222621083259583


In [46]:
rsmodel.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 156, 156,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 75, 75,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 77, 77,    │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 38, 38,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 38, 38,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 38, 38,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 38, 38,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 38, 38,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 38, 38,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 40, 40,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 38, 38,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 38, 38,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 38, 38,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 38, 38,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 38, 38,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 38, 38,    │          0 │ conv2_block1_0_c

 Total params: 66,732,548 (254.56 MB)

 Trainable params: 66,588,802 (254.02 MB)

 Non-trainable params: 143,744 (561.50 KB)

 Optimizer params: 2 (12.00 B)